In [1]:
from operator import neg
import pandas as pd
import pickle
import requests
from requests.structures import CaseInsensitiveDict
import json
import shutil
import time

In [2]:
client_id = ""
client_secret = ""

In [3]:
# Get a bearer token.
petfinder_token_url = "https://api.petfinder.com/v2/oauth2/token"

data = {'grant_type' : 'client_credentials', 'client_id' : client_id , 'client_secret' : client_secret}

response = requests.post(petfinder_token_url, data = data)

json_response = json.loads(response.content)

bearer_token = json_response['access_token']

In [4]:
# Get Best Friends Society Organization.
petfinder_get_organizations_url = "https://api.petfinder.com/v2/organizations"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

params = {'name' : 'Best Friends Animal Society'}

response = requests.get(petfinder_get_organizations_url, params = params, headers = headers)

json_response = json.loads(response.content)

for i in range(5):
    print(json_response['organizations'][i])

{'id': 'UT81', 'name': 'Best Friends Animal Society-Utah', 'email': 'utahadoptions@bestfriends.org', 'phone': '(801) 574-2454', 'address': {'address1': '2005 S 1100 E', 'address2': None, 'city': 'Salt Lake City', 'state': 'UT', 'postcode': '84106', 'country': 'US'}, 'hours': {'monday': '10 am - 7 pm', 'tuesday': '10 am - 7 pm', 'wednesday': '10 am - 7 pm', 'thursday': '10 am - 7 pm', 'friday': '10 am - 7 pm', 'saturday': '10 am - 7 pm', 'sunday': '10 am - 7 pm'}, 'url': 'https://www.petfinder.com/member/us/ut/salt-lake-city/best-friends-animal-society-utah-ut81/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'website': 'http://utah.bestfriends.org', 'mission_statement': 'The mission of Best Friends Animal Society is to bring about a time when there are No More Homeless Pets....', 'adoption': {'policy': 'Due to Covid-19, our SLC Lifesaving Center is temporarily closed. All pets are currently in foster homes and adoption meet...', 'url': 'https://utah.bestfriends.org/get-involved/ado

In [5]:
# The Best Friends Animal Society's main office is the one with id "UT77".
# Getting 100 profiles from adoptable animals from that organization.
petfinder_get_animals_url = "https://api.petfinder.com/v2/animals"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer " + bearer_token

organization_id = "UT77"
limit = 100

params = {'organization' : organization_id, 'limit' : limit}

response = requests.get(petfinder_get_animals_url, params = params, headers = headers)

json_response = json.loads(response.content)

print(json_response)

{'animals': [{'id': 59000445, 'organization_id': 'UT77', 'url': 'https://www.petfinder.com/cat/burkhart-59000445/ut/kanab/best-friends-animal-society-ut77/?referrer_id=b23ab561-58a7-4b32-b872-5418ff49f309', 'type': 'Cat', 'species': 'Cat', 'breeds': {'primary': 'Domestic Short Hair', 'secondary': None, 'mixed': False, 'unknown': False}, 'colors': {'primary': 'Gray / Blue / Silver', 'secondary': None, 'tertiary': None}, 'age': 'Adult', 'gender': 'Female', 'size': 'Small', 'coat': None, 'attributes': {'spayed_neutered': False, 'house_trained': False, 'declawed': False, 'special_needs': False, 'shots_current': True}, 'environment': {'children': None, 'dogs': None, 'cats': None}, 'tags': [], 'name': 'Burkhart', 'description': None, 'organization_animal_id': 'BKNB-A-42581', 'photos': [], 'primary_photo_cropped': None, 'videos': [], 'status': 'adoptable', 'status_changed_at': '2022-11-30T00:06:55+0000', 'published_at': '2022-11-30T00:06:54+0000', 'distance': None, 'contact': {'email': 'adopt

In [6]:
print(len(json_response['animals']))

100


In [7]:
# Harmonize data.
animals = json_response['animals']
harmonized_animals = []

for animal in animals:

    harmonized_animal = {'Type' : animal['type'], 'Name' : animal['name'], 'Age' : animal['age'],\
                           'Breed1' : animal['breeds']['primary'], 'Breed2' : animal['breeds']['secondary'], \
                           'Gender' : animal['gender'], 'Color1' : animal['colors']['primary'], \
                           'Color2' : animal['colors']['secondary'], 'Color3' : animal['colors']['tertiary'], \
                           'MaturitySize' : animal['size'], 'FurLength' : None, 'Vaccinated' : animal['attributes']['shots_current'], \
                           'Dewormed' : None, 'Sterilized' : animal['attributes']['spayed_neutered'], 'Health' : None, \
                           'Quantity' : 1, 'Fee' : None, 'State' : animal['contact']['address']['state'], 'RescuerID' : None, \
                           'VideoAmt' : len(animal['videos']), 'Description' : animal['description'], 'PetID' : animal['id'], \
                           'PhotoAmt' : len(animal['photos']), 'AdoptionSpeed' : None, 'id' : animal['id'] }
    harmonized_animals.append(harmonized_animal)

print(harmonized_animals)

[{'Type': 'Cat', 'Name': 'Burkhart', 'Age': 'Adult', 'Breed1': 'Domestic Short Hair', 'Breed2': None, 'Gender': 'Female', 'Color1': 'Gray / Blue / Silver', 'Color2': None, 'Color3': None, 'MaturitySize': 'Small', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': False, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': None, 'PetID': 59000445, 'PhotoAmt': 0, 'AdoptionSpeed': None, 'id': 59000445}, {'Type': 'Cat', 'Name': 'Rhonda', 'Age': 'Adult', 'Breed1': 'Domestic Short Hair', 'Breed2': None, 'Gender': 'Female', 'Color1': 'Torbie', 'Color2': None, 'Color3': None, 'MaturitySize': 'Small', 'FurLength': None, 'Vaccinated': True, 'Dewormed': None, 'Sterilized': False, 'Health': None, 'Quantity': 1, 'Fee': None, 'State': 'UT', 'RescuerID': None, 'VideoAmt': 0, 'Description': None, 'PetID': 59000444, 'PhotoAmt': 0, 'AdoptionSpeed': None, 'id': 59000444}, {'Type': 'Cat', 'Name': 'Degree', 'Age': 'Young', 'Breed1':

In [8]:
df = pd.DataFrame(harmonized_animals)

In [9]:
df.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,id
0,Cat,Burkhart,Adult,Domestic Short Hair,None,Female,Gray / Blue / Silver,None,None,Small,...,1,None,UT,None,0,None,59000445,0,None,59000445
1,Cat,Rhonda,Adult,Domestic Short Hair,None,Female,Torbie,None,None,Small,...,1,None,UT,None,0,None,59000444,0,None,59000444
2,Cat,Degree,Young,Domestic Short Hair,None,Male,Brown / Chocolate,None,None,Small,...,1,None,UT,None,0,A happy-go-lucky kitty that just wants a lap t...,59000441,0,None,59000441
3,Cat,Midge,Adult,Domestic Short Hair,None,Female,Gray / Blue / Silver,None,None,Small,...,1,None,UT,None,0,None,59000442,0,None,59000442
4,Cat,Pinciotti,Adult,Domestic Short Hair,None,Female,Gray / Blue / Silver,None,None,Small,...,1,None,UT,None,0,None,59000443,0,None,59000443
5,Cat,Kelso,Adult,Domestic Short Hair,None,Male,Black,None,None,Small,...,1,None,UT,None,0,None,59000439,0,None,59000439
6,Cat,Conifer,Adult,Domestic Short Hair,None,Male,Black,None,None,Small,...,1,None,UT,None,0,None,59000440,1,None,59000440
7,Dog,Yam,Baby,Black Labrador Retriever,None,Male,Black,None,None,Small,...,1,None,UT,None,0,None,58989900,3,None,58989900
8,Cat,Maryke,Young,Domestic Short Hair,None,Female,Calico,None,None,Small,...,1,None,UT,None,0,None,58987460,1,None,58987460
9,Cat,Bellagio,Adult,Domestic Short Hair,None,Male,Cream / Ivory,None,None,Small,...,1,None,UT,None,0,Are you looking to up your daily dose of laugh...,58987461,2,None,58987461


In [10]:
#df.to_csv('data/best_friends_animal_society/test.csv', index = False)

In [16]:
# Applying harmonization logic.
from petfinder_animal import petfinder_animal
fourth_animal = json_response['animals'][2]

harmonized_animal = petfinder_animal.create_harmonized_petfinder_animal(fourth_animal)

# Converting harmonized animal to dataframe.
print(harmonized_animal.as_kaggle_dictionary())
harmonized_animal_df = pd.DataFrame([harmonized_animal.as_kaggle_dictionary()])
harmonized_animal_df

{'Type': 2, 'Breed1': 0, 'Breed2': 0, 'Gender': 1, 'Color1': 0, 'Color2': 0, 'MaturitySize': 1, 'FurLength': 0, 'Vaccinated': 1, 'Dewormed': 3, 'Sterilized': 0, 'Health': 0, 'State': 0, 'HasDescription': True, 'Age': 0, 'Quantity': 1, 'Fee': 0, 'VideoAmt': 0, 'PhotoAmt': 0, 'DescriptionLength': 116, 'nltk_negative_prob': 0.0, 'nltk_neutral_prob': 0.85, 'nltk_positive_prob': 0.15, 'nltk_compound_score': 0.4588}


,Type,Breed1,Breed2,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Dewormed,...,Age,Quantity,Fee,VideoAmt,PhotoAmt,DescriptionLength,nltk_negative_prob,nltk_neutral_prob,nltk_positive_prob,nltk_compound_score
0,2,0,0,1,0,0,1,0,1,3,...,0,1,0,0,0,116,0.0,0.85,0.15,0.4588


In [12]:
# Load Model with pickle.
model = pickle.load(open('models/random_forest_classifier_kaggle.pkl','rb'))

C:\Users\oswal\anaconda3\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\oswal\anaconda3\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
# Call Prediction on dataframe.
print(model.predict(harmonized_animal_df))

[1]


C:\Users\oswal\anaconda3\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [17]:
print(harmonized_animal.Description)

A happy-go-lucky kitty that just wants a lap to fall asleep on. This sweet boy becomes an instant purring machine...


In [18]:
# Getting feedback from the animal:
from petfinder_feedback import petfinder_feedback

feedback = petfinder_feedback(harmonized_animal)
print(feedback.sentence_sentiments)
print(feedback.photo_blurriness)

{'A happy-go-lucky kitty that just wants a lap to fall asleep on.': 0.0, 'This sweet boy becomes an instant purring machine...': 0.0}
{}


In [21]:
# Check for other animals.
for animal in json_response['animals']:
    harmonized_animal = petfinder_animal.create_harmonized_petfinder_animal(animal)
    print("Feedback ")
    feedback = petfinder_feedback(harmonized_animal)
    print(feedback.sentence_sentiments)
    #print(feedback.photo_blurriness)

Feedback 
{}
Feedback 
{}
Feedback 
{'A happy-go-lucky kitty that just wants a lap to fall asleep on.': 0.0, 'This sweet boy becomes an instant purring machine...': 0.0}
Feedback 
{}
Feedback 
{}
Feedback 
{}
Feedback 
{}
Feedback 
{}
Feedback 
{}
Feedback 
{'Are you looking to up your daily dose of laughter, cuddles, and &amp;#34;Aww&amp;#34; moments?': 0.0, 'Overrun with too many ping pong...': 0.0}
Feedback 
{}
Feedback 
{'Photos of Bagel really do not do him justice - this guy is a stunner!': 0.204, 'Bagel loves attention and would...': 0.0}
Feedback 
{}
Feedback 
{}
Feedback 
{}
Feedback 
{'Don&amp;#39;t let his name fool you!': 0.39, 'This gentle giant is all cuddles and fluff!': 0.0, 'Hercules would love a low-key home...': 0.0}
Feedback 
{'Do you want to come see all my toys?': 0.0, 'Blaze is an affectionate, snuggly boy who loves to play with...': 0.0}
Feedback 
{}
Feedback 
{}
Feedback 
{'Are you looking to up your daily dose of laughter, cuddles, and &amp;#34;Aww&amp;#34; mo

{'Derby loooooves people.': 0.0, 'And he has a lot to say about a lot of things.': 0.0, 'Derby is a solid boy...': 0.0}
Feedback 
{}
Feedback 
{'Are you looking to up your daily dose of laughter, cuddles, and &amp;#34;Aww&amp;#34; moments?': 0.0, 'Overrun with too many ping pong...': 0.0}
Feedback 
{'Quench is a purrfect gentleman, and loves to play!': 0.0, 'Super sweet, playful and curious, this boy enjoys his cat friends,...': 0.0}
Feedback 
{'Are you looking for a sweet boy to brighten up your day and take up all your phone photo storage?...': 0.0}
Feedback 
{'Hi, I&amp;#39;m Vinaigrette.': 0.0, 'I love getting attention and am looking for a home where I can give my family all...': 0.0}


In [ ]:
#harmonized_animal.Photos[0]['small']
#res = requests.get(harmonized_animal.Photos[0]['small'], stream = True)
#with open("temp.jpg",'wb') as f:
#        shutil.copyfileobj(res.raw, f)
#urllib.urlretrieve(harmonized_animal.Photos[0]['small'], full_path)